# Weekly Sales to Target Tracker Function

The objective of this function is to create tabular data used for creating a Weekly Sales to Target Tracker. This one function will return two files: a 'quarter to date' CSV and an 'annual to date' CSV.

Tableau Server Team/Rep Performance Sales Table is downloaded and fed into this function. This function also takes in the date of data retrieval, the quarter number, current number of billing days in the quarter, and current number of billing days in the year.

**Note:** Starting Q2, this dataframe will also include all NTX territory.

The function knows the finalized targets for each team, and uses this in conjunction with the imported sales data to create a dataframe; it also automatically performs calculations for all columns requested:
   * QTD/YTD Target
   * % to QTD/YTD Target 
   * % Growth PY
   * % to Quarter/Annual Target
    
The function will automatically display the tables it has created, as well as download those tables as CSV files to same path as this notebook. 

From there, the Power BI file is refreshed with the new data source.

## Data Acquisition

**Team/Rep Sales Performance**

Explore / Territory Leadership / West Zone Daily Sales (2022) Territory / Team/Rep Performance / 
Download Team_Rep Sales Table_crosstab

Following filters:
* QTD and YTD 
* Financial Sales
* Product Groups: CEMENT, EXTREMITIES, FOOT AND ANKLE, HIPS, KNEES, RESTORATIVE THERAPY, SPORTS MEDICINE, TRAUMA
* Team/Rep Name: JOHN KOEHLER, MATT BORROR, RYAN FURNISH, TEAM BLAKE ROBERTS, TEAM DOSSIE LAYTON, TEAM LUNA, TEAM MIDAS, TEAM NEO, TEAM OKC, TEAM RICK ATCHISON, TEAM SEVE SANCHEZ, TEAM SOUTHWEST, TEAM TULSA
* Direct Comparison and 2021 as PY
* Today's date selected; currently these updates are occurring on Friday mornings.

.txt file from Tableau crosstab download converted to .csv UTF-8 to be read by this function.


In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
from pathlib import Path

## Target Tracker Function

In [ ]:
def target_tracker(date, q_number, q_current_billing_days, annual_current_billing_days):
    
    # Date is written 'mmddyy'
    # q_number written as string e.g.'1'
    
    # Establish quarter
    quarter_billing_days = {'1':63,'2':63,'3':64,'4':61}
    total_billing_days_quarter = quarter_billing_days[q_number]
    
    # Download final targets dataframe
    final_targets = pd.read_csv('final_targets_2022.csv')
    # Clean final targets dataframe
    final_targets.rename(columns={'TEAM': 'team_name', 'SALES MANAGER': 'sales_manager', 'PRODUCT GROUP': 'product_group', 
                              'Q1 TARGET': 'q1_target', 'Q2 TARGET': 'q2_target', 'Q3 TARGET': 'q3_target', 
                              'Q4 TARGET': 'q4_target', 'TOTAL TARGET': 'total_target', 'Q2-Q4 TARGET': 'q2_q4_target'},
                         inplace=True)
    final_targets = final_targets.replace('\$','', regex=True).replace(',','', regex=True).replace('-','0',regex=True)
    final_targets_annual = final_targets[['team_name', 'sales_manager', 'product_group', 'total_target']]
    
    # Download Tableau data
    quarterly_sales = pd.read_csv('quarterly_sales_'+date+'.csv', na_values='$0 ')
    annual_sales = pd.read_csv('annual_sales_'+date+'.csv', na_values='$0 ')
    
    # Clean quarter dataframe
    quarterly_sales.rename(columns={'Team/Rep Name': 'team_name', 'Product Group': 'product_group',
                                    'Current Sales': 'current_sales', 'Previous Sales': 'sales_2021',
                                    'Growth %': 'percent_growth_py'},
                           inplace=True)
    quarterly_sales = quarterly_sales[['team_name', 'product_group', 'current_sales', 'sales_2021']]
    quarterly_sales = quarterly_sales.replace('\$','', regex=True).replace(',','', regex=True)
    quarterly_sales['product_group'] = quarterly_sales['product_group'].str.replace('&','AND',regex=True)
    quarterly_sales['product_group'] = quarterly_sales['product_group'].str.upper()
    quarterly_sales[['current_sales', 'sales_2021']] = quarterly_sales[['current_sales', 'sales_2021']].astype(float)
    
    # Merge quarter dataframe with final targets
    qtr_merged = final_targets.merge(quarterly_sales, how='left', on=['team_name','product_group'])
    qtr_merged = qtr_merged[qtr_merged['product_group']!='TOTAL']
    
    # Quarter target/percentage calculations
    qtr_merged['qtd_target'] = ((abs(qtr_merged['q'+q_number+'_target']))/total_billing_days_quarter)*q_current_billing_days
    qtr_merged['percent_to_qtd_target'] = qtr_merged['current_sales']/qtr_merged['qtd_target']
    qtr_merged['percent_growth_py'] = (qtr_merged['current_sales']-qtr_merged['sales_2021'])/qtr_merged['sales_2021']
    qtr_merged['percent_to_qtr_target'] = qtr_merged['current_sales']/qtr_merged['q'+q_number+'_target']
    
    # Repeat for annual dataframe
    annual_sales.rename(columns={'Team/Rep Name': 'team_name', 'Product Group': 'product_group',
                                    'Current Sales': 'current_sales', 'Previous Sales': 'sales_2021',
                                    'Growth %': 'percent_growth_py'},
                           inplace=True)
    annual_sales = annual_sales[['team_name', 'product_group', 'current_sales', 'sales_2021']]
    annual_sales = annual_sales.replace('\$','', regex=True).replace(',','', regex=True)
    annual_sales['product_group'] = annual_sales['product_group'].str.replace('&','AND',regex=True)
    annual_sales['product_group'] = annual_sales['product_group'].str.upper()
    annual_sales[['current_sales', 'sales_2021']] = annual_sales[['current_sales', 'sales_2021']].astype(float)
    
    # Merge annual dataframe with final targets
    annual_merged = final_targets_annual.merge(annual_sales, how='left', on=['team_name','product_group'])
    annual_merged = annual_merged[annual_merged['product_group']!='TOTAL']
    
    # Annual target/percentage calculations
    annual_merged['ytd_target'] = ((abs(annual_merged['total_target']))/251)*annual_current_billing_days
    annual_merged['percent_to_ytd_target'] = annual_merged['current_sales']/annual_merged['ytd_target']
    annual_merged['percent_growth_py'] = (annual_merged['current_sales']-annual_merged['sales_2021'])/annual_merged['sales_2021']
    annual_merged['percent_to_year_target'] = annual_merged['current_sales']/annual_merged['total_target']
    
    # Download both new dataframes
    #quarter_filepath = Path(r'Unavailable')
    #quarter_filepath.parent.mkdir(parents=True, exist_ok=True)
    #qtr_merged.to_csv(quarter_filepath, index=False)
    #Above code not necessary if notebook stored in same location as report and data
    qtr_merged.to_csv('quarter_dash_'+date+'.csv', index=False)
    
    #annual_filepath = Path(r'Unavailable')
    #annual_filepath.parent.mkdir(parents=True, exist_ok=True)
    #annual_merged.to_csv(annual_filepath, index=False)
    #Above code not necessary if notebook stored in same location as report and data                  
    annual_merged.to_csv('annual_dash_'+date+'.csv', index=False)
    
    return qtr_merged, annual_merged

In [ ]:
target_tracker('031122', '1', 48, 48)

**Development Notes**


* str.replace updating soon; must include regex=True [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html#pandas.Series.str.replace)